<a href="https://colab.research.google.com/github/spazewalker/SpeechRecog-Opencv/blob/master/Jasper2FP32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --content-disposition --quiet https://api.ngc.nvidia.com/v2/models/nvidia/jasper_pyt_onnx_fp16_amp/versions/20.10.0/zip -O jasper_pyt_onnx_fp16_amp_20.10.0.zip
!unzip -o ./jasper_pyt_onnx_fp16_amp_20.10.0.zip
!unzip -o ./jasper_pyt_onnx_fp16_amp.zip
!pip install onnx

Archive:  ./jasper_pyt_onnx_fp16_amp_20.10.0.zip
  inflating: jasper_pyt_onnx_fp16_amp.zip  
Archive:  ./jasper_pyt_onnx_fp16_amp.zip
  inflating: decoder-ts-script/1/model.pt  
  inflating: feature-extractor-ts-trace/1/model.pt  
  inflating: jasper-onnx/1/model.onnx  


In [2]:
import onnx
model = onnx.load("./jasper-onnx/1/model.onnx")
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %input__0[FLOAT16, input__0_dynamic_axes_1x64xinput__0_dynamic_axes_2]
) initializers (
  %decoder.layers.0.bias[FLOAT16, 29]
  %decoder.layers.0.weight[FLOAT16, 29x1024x1]
  %encoder.layers.0.conv.0.weight[FLOAT16, 256x64x11]
  %encoder.layers.0.conv.1.bias[FLOAT16, 256]
  %encoder.layers.0.conv.1.running_mean[FLOAT16, 256]
  %encoder.layers.0.conv.1.running_var[FLOAT16, 256]
  %encoder.layers.0.conv.1.weight[FLOAT16, 256]
  %encoder.layers.1.conv.0.weight[FLOAT16, 256x256x11]
  %encoder.layers.1.conv.1.bias[FLOAT16, 256]
  %encoder.layers.1.conv.1.running_mean[FLOAT16, 256]
  %encoder.layers.1.conv.1.running_var[FLOAT16, 256]
  %encoder.layers.1.conv.1.weight[FLOAT16, 256]
  %encoder.layers.1.conv.12.weight[FLOAT16, 256x256x11]
  %encoder.layers.1.conv.13.bias[FLOAT16, 256]
  %encoder.layers.1.conv.13.running_mean[FLOAT16, 256]
  %encoder.layers.1.conv.13.running_var[FLOAT16, 256]
  %encoder.layers.1.conv.13.weight[FLOAT16, 256]
  %encoder.layers.1.conv.16.

In [3]:
for node_tmp in model.graph.node:
    print(node_tmp)

input: "input__0"
input: "encoder.layers.0.conv.0.weight"
output: "651"
name: "Conv_0"
op_type: "Conv"
attribute {
  name: "dilations"
  ints: 1
  type: INTS
}
attribute {
  name: "group"
  i: 1
  type: INT
}
attribute {
  name: "kernel_shape"
  ints: 11
  type: INTS
}
attribute {
  name: "pads"
  ints: 5
  ints: 5
  type: INTS
}
attribute {
  name: "strides"
  ints: 2
  type: INTS
}

input: "651"
input: "encoder.layers.0.conv.1.weight"
input: "encoder.layers.0.conv.1.bias"
input: "encoder.layers.0.conv.1.running_mean"
input: "encoder.layers.0.conv.1.running_var"
output: "652"
name: "BatchNormalization_1"
op_type: "BatchNormalization"
attribute {
  name: "epsilon"
  f: 0.0010000000474974513
  type: FLOAT
}
attribute {
  name: "momentum"
  f: 0.8999999761581421
  type: FLOAT
}

input: "652"
output: "653"
name: "Relu_2"
op_type: "Relu"

input: "653"
input: "encoder.layers.1.conv.0.weight"
output: "654"
name: "Conv_3"
op_type: "Conv"
attribute {
  name: "dilations"
  ints: 1
  type: INTS


In [4]:
for i in range(16):
    print(i,onnx.TensorProto.DataType.Name(i))
# 17 data types. Input is of type Float16. should change it to Float i.e. 1

0 UNDEFINED
1 FLOAT
2 UINT8
3 INT8
4 UINT16
5 INT16
6 INT32
7 INT64
8 STRING
9 BOOL
10 FLOAT16
11 DOUBLE
12 UINT32
13 UINT64
14 COMPLEX64
15 COMPLEX128


In [5]:
# Change input data type FLOAT16 ==> FLOAT
inp = model.graph.input[0]
model.graph.input.remove(inp)
inp.type.tensor_type.elem_type = 1
model.graph.input.insert(0,inp)

In [6]:
# Change output data type FLOAT16 ==> FLOAT
out = model.graph.output[0]
model.graph.output.remove(out)
out.type.tensor_type.elem_type = 1
model.graph.output.insert(0,out)

In [7]:
model.graph.input

[name: "input__0"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_param: "input__0_dynamic_axes_1"
      }
      dim {
        dim_value: 64
      }
      dim {
        dim_param: "input__0_dynamic_axes_2"
      }
    }
  }
}
]

In [8]:
# Change data type FLOAT16 ==> FLOAT of every initilizer
for i,init in enumerate(model.graph.initializer):
    model.graph.initializer.remove(init)
    init.data_type = 1
    model.graph.initializer.insert(i,init)

In [9]:
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %input__0[FLOAT, input__0_dynamic_axes_1x64xinput__0_dynamic_axes_2]
) initializers (
  %decoder.layers.0.bias[FLOAT, 29]
  %decoder.layers.0.weight[FLOAT, 29x1024x1]
  %encoder.layers.0.conv.0.weight[FLOAT, 256x64x11]
  %encoder.layers.0.conv.1.bias[FLOAT, 256]
  %encoder.layers.0.conv.1.running_mean[FLOAT, 256]
  %encoder.layers.0.conv.1.running_var[FLOAT, 256]
  %encoder.layers.0.conv.1.weight[FLOAT, 256]
  %encoder.layers.1.conv.0.weight[FLOAT, 256x256x11]
  %encoder.layers.1.conv.1.bias[FLOAT, 256]
  %encoder.layers.1.conv.1.running_mean[FLOAT, 256]
  %encoder.layers.1.conv.1.running_var[FLOAT, 256]
  %encoder.layers.1.conv.1.weight[FLOAT, 256]
  %encoder.layers.1.conv.12.weight[FLOAT, 256x256x11]
  %encoder.layers.1.conv.13.bias[FLOAT, 256]
  %encoder.layers.1.conv.13.running_mean[FLOAT, 256]
  %encoder.layers.1.conv.13.running_var[FLOAT, 256]
  %encoder.layers.1.conv.13.weight[FLOAT, 256]
  %encoder.layers.1.conv.16.weight[FLOAT, 256x256x11]
  %encoder

In [10]:
with open('jasper_dynamic_input_float.onnx','wb') as f:
    onnx.save_model(model,f)
model = onnx.load('jasper_dynamic_input_float.onnx')

In [11]:
inp = model.graph.input[0]
model.graph.input.remove(inp)
inp.type.tensor_type.shape.dim[2].dim_value = 256
inp.type.tensor_type.shape.dim[0].dim_value = 1
model.graph.input.insert(0,inp)

In [12]:
model.graph.input

[name: "input__0"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 64
      }
      dim {
        dim_value: 256
      }
    }
  }
}
]

In [13]:
with open('jasper_input_1x64x256_float.onnx','wb') as f:
    onnx.save_model(model,f)
model = onnx.load('jasper_input_1x64x256_float.onnx')

In [14]:
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %input__0[FLOAT, 1x64x256]
) initializers (
  %decoder.layers.0.bias[FLOAT, 29]
  %decoder.layers.0.weight[FLOAT, 29x1024x1]
  %encoder.layers.0.conv.0.weight[FLOAT, 256x64x11]
  %encoder.layers.0.conv.1.bias[FLOAT, 256]
  %encoder.layers.0.conv.1.running_mean[FLOAT, 256]
  %encoder.layers.0.conv.1.running_var[FLOAT, 256]
  %encoder.layers.0.conv.1.weight[FLOAT, 256]
  %encoder.layers.1.conv.0.weight[FLOAT, 256x256x11]
  %encoder.layers.1.conv.1.bias[FLOAT, 256]
  %encoder.layers.1.conv.1.running_mean[FLOAT, 256]
  %encoder.layers.1.conv.1.running_var[FLOAT, 256]
  %encoder.layers.1.conv.1.weight[FLOAT, 256]
  %encoder.layers.1.conv.12.weight[FLOAT, 256x256x11]
  %encoder.layers.1.conv.13.bias[FLOAT, 256]
  %encoder.layers.1.conv.13.running_mean[FLOAT, 256]
  %encoder.layers.1.conv.13.running_var[FLOAT, 256]
  %encoder.layers.1.conv.13.weight[FLOAT, 256]
  %encoder.layers.1.conv.16.weight[FLOAT, 256x256x11]
  %encoder.layers.1.conv.17.bias[FLOAT, 256]
  %enco

In [15]:
!pip install opencv-python==4.5.2.52
import cv2 as cv
# Should be > 4.5.x
cv.__version__

'4.5.2'

In [16]:
import cv2 as cv
import numpy as np
net = cv.dnn.readNetFromONNX('./jasper_input_1x64x256_float.onnx') # N,C,W
net1 = cv.dnn.readNetFromONNX('./jasper_dynamic_input_float.onnx')

In [17]:
input_img = np.random.randn(320,64,32)

In [18]:
net1.setInput(input_img)
out = net1.forward()

error: ignored

In [ ]:
import torch
encoder = torch.jit.load('feature-extractor-ts-trace/1/model.pt')

In [ ]:
encoder

In [ ]:
decoder = torch.jit.load('decoder-ts-script/1/model.pt')

In [ ]:
decoder